# ロボットマニピュレータの逆運動学を4×4マトリクスを用いて解く！！！

In [1]:
#参考URL -> https://qiita.com/tibigame/items/61cecf86fc978628bfee
#参考図書 -> ポールのロボット・マニピュレータ

import numpy as np
import sympy as sym
sym.init_printing()
Pi = sym.S.Pi # 円周率
E = sym.S.Exp1 # 自然対数の底
I = sym.S.ImaginaryUnit # 虚数単位

#sympyの円周率の方を使うことをすすめる（こっちの方が量子化誤差が大きくなる．numpyも同様に大きい）
import math
pi = math.pi

# 使用する変数の定義(小文字1文字は全てシンボルとする)
(a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z) = sym.symbols('a b c d e f g h i j k l m n o p q r s t u v w x y z')
#a～fは角度変数

(n_x, n_y, n_z, o_x, o_y, o_z, a_x, a_y, a_z, p_x, p_y, p_z) = sym.symbols('n_x, n_y, n_z, o_x, o_y, o_z, a_x, a_y, a_z, p_x, p_y, p_z')


In [2]:
#sin.cosの簡易記述用
def S(a):
    return sym.sin(a)

def C(a):
    return sym.cos(a)

In [3]:
#回転・並進行列
def rotx(a):
    return sym.Matrix([[1, 0, 0, 0], [0, C(a), -S(a), 0], [0, S(a), C(a), 0], [0, 0, 0, 1]])

def roty(a):
    return sym.Matrix([[C(a), 0, S(a), 0], [0, 1, 0, 0], [-S(a), 0, C(a), 0], [0, 0, 0, 1]])

def rotz(a):
    return sym.Matrix([[C(a), -S(a), 0, 0], [S(a), C(a), 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])

def trans(x, y, z):
    return sym.Matrix([[1, 0, 0, x], [0, 1, 0, y], [0, 0, 1, z], [0, 0, 0, 1]])


### DH法に基づく座標系間の関係表現

|座標系 i|Z_i-1軸回りに角度θ_i|Z_i-1に沿って距離d_iだけ並進|回転後のX_i-1 (=X_i)に沿って長さa_iだけ並進|X_i軸周りにねじれ角α_iだけ回転|
|-|-|-|-|-|
|1|a|h|g|Pi/2|
|2|b|0|0|0|(yにj)
|3|c|0|-k|-Pi/2|(cの初期位置は-Pi/2)
|4|d|l|0|Pi/2|
|5|e|0|0|-Pi/2|(yにm)
|6|f|n|0|0|

座標系0→1：
Z0軸方向にd1だけ平行移動してから、関節１の関節角度θ1だけ回転、a1=0、α1=0

座標系1→2：
X1軸方向にa2移動、関節２の角度θ2だけ回転、α2=0、d2=0

座標系2→3：
X2軸方向にa3移動、X2軸回りにα3=180度(π)回転、回転後のZ2軸方向(鉛直下向)にd3移動、θ3=0

座標系3→4：
Z0軸方向にd1だけ平行移動してから、関節１の関節角度θ1だけ回転、a1=0、α1=0

座標系4→5：
X1軸方向にa2移動、関節２の角度θ2だけ回転、α2=0、d2=0

座標系5→6：
X2軸方向にa3移動、X2軸回りにα3=180度(π)回転、回転後のZ2軸方向(鉛直下向)にd3移動、θ3=0

In [ ]:
#変換行列の指定（1.モーターによる回転成分 -> 2.機構に基づく並進成分 -> 3.機構に基づく座標系の回転）

A1=rotz(a)*trans(g,0,h)*rotx(Pi/2)
sym.trigsimp(A1)

In [ ]:
#変換行列の指定（1.モーターによる回転成分 -> 2.機構に基づく並進成分 -> 3.機構に基づく座標系の回転）

A2=rotz(b)*trans(0,j,0)
sym.trigsimp(A2)

In [ ]:
#変換行列の指定（1.モーターによる回転成分 -> 2.機構に基づく並進成分 -> 3.機構に基づく座標系の回転）

A3=rotz(c)*trans(0,k,0)*roty(Pi/2)
sym.trigsimp(A3)

In [ ]:
#変換行列の指定（1.モーターによる回転成分 -> 2.機構に基づく並進成分 -> 3.機構に基づく座標系の回転）

A4=rotz(d)*trans(0,0,l)*roty(-Pi/2)
sym.trigsimp(A4)

In [ ]:
#変換行列の指定（1.モーターによる回転成分 -> 2.機構に基づく並進成分 -> 3.機構に基づく座標系の回転）

A5=rotz(e)*trans(m,0,0)*roty(Pi/2)
sym.trigsimp(A5)

In [ ]:
#変換行列の指定（1.モーターによる回転成分 -> 2.機構に基づく並進成分 -> 3.機構に基づく座標系の回転）

A6=rotz(f)*trans(0,0,n)
sym.trigsimp(A6)

In [ ]:
sym.trigsimp(A1*A2*A3*A4*A5*A6)

In [ ]:
T5=sym.trigsimp(A6)
T4=sym.trigsimp(A5*A6)
T3=sym.trigsimp(A4*A5*A6)
T2=sym.trigsimp(A3*A4*A5*A6)
T1=sym.trigsimp(A2*A3*A4*A5*A6)
T=sym.trigsimp(A1*A2*A3*A4*A5*A6)

#x,y,zの順運動学が確認したければここを表示して，Tの４列目を確認
#sym.trigsimp(T*trans(n,0,0))

In [ ]:
B1=A1.inv()
B2=A2.inv()
B3=A3.inv()
B4=A4.inv()
B5=A5.inv()
B6=A6.inv()

In [ ]:
#逆行列確認
#sym.trigsimp(A1*B1*A2*B2*A3*B3*A4*B4*A5*B5*A6*B6*A7*B7)

In [ ]:
U1=sym.trigsimp(B1)
U2=sym.trigsimp(B2*B1)
U3=sym.trigsimp(B3*B2*B1)
U4=sym.trigsimp(B4*B3*B2*B1)
U5=sym.trigsimp(B5*B4*B3*B2*B1)

In [ ]:
#T.subs([(a,0),(b,0),(c,0),(d,0),(e,0),(f,0)])

In [ ]:
#T=sym.Matrix([[n_x, o_x, a_x, p_x], [n_y, o_y, a_y, p_y], [n_z, o_z, a_z, p_z], [0, 0, 0, 1]])

In [ ]:
sym.trigsimp([U1,T1])

In [ ]:
sym.trigsimp([U2,T2])

In [ ]:
sym.trigsimp([U3,T3])

In [ ]:
sym.trigsimp([U4,T4])

In [ ]:
sym.trigsimp([U5,T5])

In [ ]:
#x=T[0,3]
#y=T[1,3]
#z=T[2,3]